# interaction_builder

In [ ]:
#| default_exp interaction_builder

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from tk_slack.core import ValueFormatter

from fastcore.basics import patch_to
from fastcore.test import *

from typing import List, Tuple, Dict, Any, Callable, Optional

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

import pandas as pd
import json, re, os
from datetime import datetime
import pytz
import time
import random
import uuid

# SnowflakeConnector

Connector class for Snowflake operations related to Slack interactions. Handles connecting to Snowflake and provides methods for data operations.

In [ ]:
#| export

class SnowflakeConnector:
    """
    Connector class for Snowflake operations related to Slack interactions.
    Handles connecting to Snowflake and provides methods for data operations.
    """
    
    def __init__(self, connection_params: Optional[Dict[str, Any]] = None):
        """Initialize the Snowflake connector.
        
        Args:
            connection_params: Dictionary with connection parameters.
                If None, will use environment variables.
        """
        # Use provided params or get from environment
        if connection_params:
            self.connection_params = connection_params
        else:
            self.connection_params = {
                'account': os.environ.get('SNOWFLAKE_ACCOUNT'),
                'user': os.environ.get('SNOWFLAKE_USER'),
                'password': os.environ.get('SNOWFLAKE_PASSWORD'),
                'warehouse': os.environ.get('SNOWFLAKE_WAREHOUSE'),
                'database': os.environ.get('SNOWFLAKE_DATABASE'),
                'schema': os.environ.get('SNOWFLAKE_SCHEMA')
            }
        
        # Connection is initialized lazily when needed
        self._conn = None
        self._schema_cache = {}
        self.database = self.connection_params['database']
        self.schema = self.connection_params['schema']  

First, if we want to make a Snowfalke Connection class, we should probably make a way to connect to Snowflake...

In [ ]:
#| export

@patch_to(SnowflakeConnector)
def _get_connection(self):
    """Get a Snowflake connection, creating one if needed."""
    if not self._conn:
        self._conn = snowflake.connector.connect(
            **self.connection_params
        )
    return self._conn

Yay! We are now able to connect to Snowflake which is pretty cool. Let's also make a method to close the connection when we are done with it. This is important because we don't want to leave connections open and use up resources. We can do this by adding a `close` method to our class. This method will call the `close` method on the connection object. We can also add a check to see if the connection is already closed before trying to close it again.

In [ ]:
#| export
@patch_to(SnowflakeConnector)
def close(self):
        """Close the Snowflake connection if open."""
        if self._conn:
            self._conn.close()
            self._conn = None

Awesome, now we can connect to Snowflake and close the connection when we are done with it. Let's also add a method to execute a query and return the results. This will be useful for getting data from Snowflake. We can do this by adding an `execute_query` method to our class. This method will take a query string as an argument and return the results of the query. We can use the `cursor` object to execute the query and fetch the results.

In [ ]:
#| export
@patch_to(SnowflakeConnector)
def execute_query(self, query: str, params: Optional[List[Any]] = None) -> List[Dict[str, Any]]:
    """Execute a query and return results as a list of dictionaries.
    
    Args:
        query: SQL query to execute
        params: Optional parameters for the query
        
    Returns:
        List of dictionaries with query results
    """
    try:
        conn = self._get_connection()
        cursor = conn.cursor()
        
        # Execute the query
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        
        # Get column names
        columns = [desc[0] for desc in cursor.description]
        
        # Convert results to list of dictionaries
        results = []
        for row in cursor:
            results.append(dict(zip(columns, row)))
        
        return results
        
    except Exception as e:
        print(f"Error executing query: {e}")
        raise

## Data Insertion

Let's also create a way to insert data into Snowflake.

In [ ]:
#| export

@patch_to(SnowflakeConnector)
def _get_table_schema(self, table_name: str, use_cache: bool = True) -> Dict[str, str]:
    """
    Query Snowflake to get the schema of a table.
    
    Args:
        table_name: Name of the table
        use_cache: Whether to use cached schema information
        
    Returns:
        Dictionary mapping column names to their data types
    """
    # Use instance values as defaults if not provided
    database = self.database
    schema = self.schema
    
    if not database or not schema:
        raise ValueError("Database and schema must be provided")
    
    # Check cache first
    cache_key = f"{database}.{schema}.{table_name}"
    if use_cache and cache_key in self._schema_cache:
        return self._schema_cache[cache_key]
        
    schema_query = f"""
        SELECT 
            COLUMN_NAME, 
            DATA_TYPE 
        FROM 
            {database}.INFORMATION_SCHEMA.COLUMNS 
        WHERE 
            TABLE_SCHEMA = '{schema}' 
            AND TABLE_NAME = '{table_name}'
    """
    
    try:
        conn = self._get_connection()
        with conn.cursor() as cs:
            cs.execute(schema_query)
            results = cs.fetchall()
            result_schema = {row[0]: row[1] for row in results}
            
            # Cache the result
            if use_cache:
                self._schema_cache[cache_key] = result_schema
                
            return result_schema
    except Exception as e:
        print(f"Error getting schema: {e}")
        raise RuntimeError(f"Failed to get schema for {table_name}: {str(e)}")


@patch_to(SnowflakeConnector)
def _get_current_timestamp(self, timezone: str = 'America/Chicago') -> str:
    """Get current timestamp in the specified timezone."""
    tz = pytz.timezone(timezone)
    return datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

@patch_to(SnowflakeConnector)
def _create_schema_mapping(self, db_schema: Dict) -> Dict[str, str]:
    """Create a case-insensitive mapping of schema columns."""
    return {k.upper(): k for k in db_schema.keys()}


@patch_to(SnowflakeConnector)
def _prepare_insert_data(
        self,
        insert_data: Dict, 
        db_schema: Dict,
        schema_keys_map: Dict[str, str],
        auto_timestamp: bool = True,
        include_all_columns: bool = False,
        timezone: str = 'America/Chicago',
        debug: bool = False
    ) -> Dict:
    """
    Prepare and filter data for insertion based on schema.
    """
    filtered_data = {}
    current_time = self._get_current_timestamp(timezone)
    
    if include_all_columns:
        for schema_col in db_schema.keys():
            filtered_data[schema_col] = None
    
    for k, v in insert_data.items():
        k_upper = k.upper()
        if k_upper in schema_keys_map:
            # Use the actual case from the schema
            original_case_key = schema_keys_map[k_upper]
            filtered_data[original_case_key] = v
    
    if auto_timestamp:
        timestamp_fields = {
            'UPDATED_AT': 'updated_at',
            'CREATED_AT': 'created_at',
            'RUN_TIME': 'run_time',
            'INSERTED_AT': 'inserted_at',
            'TIMESTAMP': 'timestamp'
        }
        
        for upper_field, lower_field in timestamp_fields.items():
            if upper_field in schema_keys_map:
                original_case = schema_keys_map[upper_field]
                
                field_provided = False
                for k in insert_data.keys():
                    if k.upper() == upper_field:
                        field_provided = True
                        break
                
                if not field_provided:
                    filtered_data[original_case] = current_time
    
    if debug and not filtered_data:
        print(f"Available columns in schema: {list(db_schema.keys())}")
        print(f"Provided keys: {list(insert_data.keys())}")
        print(f"Case-insensitive mapping: {schema_keys_map}")
        
    return filtered_data


@patch_to(SnowflakeConnector)
def _build_insert_query(self, table_name: str, columns: List[str]) -> str:
    """Build the base INSERT query for Snowflake."""
    return f"""
        INSERT INTO {self.database}.{self.schema}.{table_name} (
            {', '.join(columns)}
        )
        SELECT
    """

@patch_to(SnowflakeConnector)
def _prepare_values_and_expressions(
    self,
    filtered_data: Dict, 
    columns: List[str], 
    db_schema: Dict
) -> Tuple[List, List[str]]:
    """Prepare values and SQL expressions for the query."""
    values = []
    select_exprs = []
    
    for col in columns:
        value = filtered_data[col]
        data_type = db_schema[col].upper()
        
        # Handle nulls consistently
        if value is None:
            select_exprs.append("NULL")
            continue
            
        if any(array_type in data_type for array_type in ['ARRAY', 'LIST']):
            if isinstance(value, list) and len(value) == 0:
                select_exprs.append("ARRAY_CONSTRUCT()")
            elif isinstance(value, list):
                placeholders = ", ".join(["%s"] * len(value))
                select_exprs.append(f"ARRAY_CONSTRUCT({placeholders})")
                values.extend(value)
            else:
                # Handle single value as array
                select_exprs.append("ARRAY_CONSTRUCT(%s)")
                values.append(value)
        
        elif any(obj_type in data_type for obj_type in ['OBJECT', 'VARIANT', 'JSON']):
            # Handle JSON object with PARSE_JSON
            select_exprs.append("PARSE_JSON(%s)")
            values.append(json.dumps(value) if value is not None else None)
        
        else:
            select_exprs.append("%s")
            values.append(value)
    
    return values, select_exprs


@patch_to(SnowflakeConnector)
def _execute_insert_query(
    self,
    database: str,
    schema: str,
    query: str,
    values: List,
    debug: bool = False
) -> bool:
    """Execute the INSERT query on Snowflake."""
    try:
        if debug:
            print("DEBUG SQL:", query)
            print("DEBUG values:", values)
            print("DEBUG values count:", len(values))
            
        conn = self._get_connection()
        with conn.cursor() as cs:
            cs.execute(query, values)
            conn.commit()
        
        return True
    
    except Exception as e:
        print(f"Error inserting data: {e}")
        if debug:
            print(f"Failed SQL: {query}")
            print(f"Values: {values}")
        raise RuntimeError(f"Failed to insert data: {str(e)}")


@patch_to(SnowflakeConnector)
def insert_record(self, table_name: str, data: Dict[str, Any], **kwargs) -> Any:
    """
    Enhanced function to insert data into Snowflake with improved type handling.
    
    Args:
        table_name: Name of the table to insert into
        data: Dictionary of data to insert
        **kwargs: Additional options for insertion
        
    Returns:
        ID of the inserted record (if id_field is provided) or True if successful
    """
    # Set default options
    options = {
        'database': self.database,
        'schema': self.schema,
        'auto_timestamp': True,
        'timezone': 'America/Chicago',
        'debug': False,
        'include_all_columns': False
    }
    
    # Update with any provided options
    options.update(kwargs)
    
    if not options['database'] or not options['schema']:
        raise ValueError("Database and schema must be provided")
    
    # Get table schema
    db_schema = self._get_table_schema(table_name)
    
    if not db_schema:
        raise ValueError(f"Could not retrieve schema for {options['database']}.{options['schema']}.{table_name}")
        
    if options['debug']:
        print(f"Table schema: {list(db_schema.keys())}")
        
    # Create case-insensitive mapping
    schema_keys_map = self._create_schema_mapping(db_schema)
    
    # Prepare and filter data
    filtered_data = self._prepare_insert_data(
        data,
        db_schema,
        schema_keys_map,
        options['auto_timestamp'],
        options['include_all_columns'],
        options['timezone'],
        options['debug']
    )
    
    # Nothing to insert
    if not filtered_data:
        print("Warning: No valid columns provided for insert")
        return False
    
    # Build the query
    columns = list(filtered_data.keys())
    insert_query = self._build_insert_query(table_name,columns)
    
    # Prepare values and expressions
    values, select_exprs = self._prepare_values_and_expressions(
        filtered_data, 
        columns, 
        db_schema
    )
    
    # Complete the query
    insert_query += ", ".join(select_exprs)
    
    self._execute_insert_query(
        options['database'],
        options['schema'],
        insert_query,
        values,
        options['debug']
    )
    
    return True

Now that we can insert individual rows, let's also add a method to insert multiple rows at once

In [ ]:
#| export

@patch_to(SnowflakeConnector)
def bulk_insert(self, table_name: str, df: pd.DataFrame, **kwargs) -> bool:
    """
    Enhanced function to bulk insert DataFrame data into Snowflake with improved type handling.
    
    Args:
        table_name: Name of the table to insert into
        df: DataFrame with columns matching table structure
        **kwargs: Additional options for insertion
            - database: Override default database
            - schema: Override default schema
            - auto_timestamp: Add timestamps to standard timestamp fields (default: True)
            - timezone: Timezone for timestamps (default: 'America/Chicago')
            - debug: Enable debug output (default: False)
            - batch_size: Rows per batch (default: 10000)
            - quote_identifiers: Whether to quote identifiers (default: False)
            - chunk_size: Size of DataFrame chunks to process (default: None)
            - auto_create_table: Create table if not exists (default: False)
            - use_column_mapping: Map DataFrame columns to table columns (default: True)
        
    Returns:
        True if successful
    """
    # Set default options
    options = {
        'database': self.database,
        'schema': self.schema,
        'auto_timestamp': True,
        'timezone': 'America/Chicago',
        'debug': False,
        'batch_size': 10000,
        'quote_identifiers': False,
        'chunk_size': None,
        'auto_create_table': False,
        'use_column_mapping': True
    }
    
    # Update with any provided options
    options.update(kwargs)
    
    if not options['database'] or not options['schema']:
        raise ValueError("Database and schema must be provided")

    # Get table schema
    try:
        db_schema = self._get_table_schema(table_name)
    except Exception as e:
        if options['debug']:
            print(f"Error getting schema: {e}")
        if options['auto_create_table']:
            # Auto-create table logic would go here
            # Not implemented in this version
            raise NotImplementedError("Auto-create table not implemented yet")
        else:
            raise ValueError(f"Could not retrieve schema for {options['database']}.{options['schema']}.{table_name}")

    if options['debug']:
        print(f"Table schema: {list(db_schema.keys())}")
        print(f"DataFrame columns: {list(df.columns)}")

    # Create case-insensitive mapping
    schema_keys_map = self._create_schema_mapping(db_schema)
    
    # Process DataFrame to match schema if needed
    if options['use_column_mapping']:
        df_processed = self._prepare_dataframe(
            df,
            db_schema,
            schema_keys_map,
            options['auto_timestamp'],
            options['timezone'],
            options['debug']
        )
    else:
        df_processed = df.copy()
    
    if df_processed.empty:
        print("Warning: No valid data to insert after processing")
        return False
    
    try:
        conn = self._get_connection()
        
        # Set database and schema context
        qualified_table = f"{options['database']}.{options['schema']}.{table_name}"
        
        if options['debug']:
            print(f"Inserting into: {qualified_table}")
            print(f"Processed DataFrame shape: {df_processed.shape}")
        
        # Use the Snowflake Pandas integration
        success, num_chunks, num_rows, output = write_pandas(
            conn=conn,
            df=df_processed,
            table_name=qualified_table,
            quote_identifiers=options['quote_identifiers'],
            chunk_size=options['chunk_size'],
            compression='gzip',  # Usually a good default
            parallel=4,          # Use parallel processing
            overwrite=False,     # Append mode
            auto_create_table=False  # We handle schema validation separately
        )
        
        if success:
            #print(f"Bulk insert into {qualified_table}: {num_rows} rows in {num_chunks} chunks")
            return True
        else:
            print(f"Bulk insert failed: {output}")
            return False
            
    except Exception as e:
        if options['debug']:
            print(f"Error bulk inserting records: {e}")
            print(f"DataFrame columns: {list(df_processed.columns)}")
            print(f"First row: {df_processed.iloc[0].to_dict() if not df_processed.empty else 'Empty DataFrame'}")
            
        raise RuntimeError(f"Failed to bulk insert data: {str(e)}")


@patch_to(SnowflakeConnector)
def _prepare_dataframe(
        self,
        df: pd.DataFrame,
        db_schema: Dict,
        schema_keys_map: Dict[str, str],
        auto_timestamp: bool = True,
        timezone: str = 'America/Chicago',
        debug: bool = False
    ) -> pd.DataFrame:
    """
    Prepare DataFrame for insertion based on table schema.
    
    Args:
        df: Input DataFrame
        db_schema: Table schema dictionary
        schema_keys_map: Case-insensitive column mapping
        auto_timestamp: Whether to add timestamps automatically
        timezone: Timezone for timestamps
        debug: Enable debug output
        
    Returns:
        Processed DataFrame ready for insertion
    """
    # Create a new DataFrame with correctly cased columns
    result_df = pd.DataFrame()
    current_time = self._get_current_timestamp(timezone)
    
    # Add all schema columns that exist in input DataFrame
    for col in db_schema.keys():
        col_upper = col.upper()
        
        # Check if column exists in input DataFrame (case-insensitive)
        matching_cols = [c for c in df.columns if c.upper() == col_upper]
        
        if matching_cols:
            # Use the first matching column from input
            input_col = matching_cols[0]
            result_df[col] = df[input_col].copy()
            
            # Handle special types based on schema
            data_type = db_schema[col].upper()
            
            # Convert array types
            if any(array_type in data_type for array_type in ['ARRAY', 'LIST']):
                # Ensure values are properly formatted as lists
                result_df[col] = result_df[col].apply(
                    lambda x: x if (isinstance(x, list) or x is None) else [x]
                )
                
            # Convert object/JSON types
            elif any(obj_type in data_type for obj_type in ['OBJECT', 'VARIANT', 'JSON']):
                # Convert values to JSON strings
                result_df[col] = result_df[col].apply(
                    lambda x: json.dumps(x) if (x is not None) else None
                )
    
    # Add timestamp columns if requested
    if auto_timestamp:
        timestamp_fields = {
            'UPDATED_AT': 'updated_at',
            'CREATED_AT': 'created_at',
            'RUN_TIME': 'run_time',
            'INSERTED_AT': 'inserted_at',
            'TIMESTAMP': 'timestamp'
        }
        
        for upper_field, lower_field in timestamp_fields.items():
            if upper_field in schema_keys_map:
                original_case = schema_keys_map[upper_field]
                
                # Skip if column already exists in result DataFrame
                if original_case in result_df.columns:
                    continue
                    
                # Add timestamp column
                result_df[original_case] = current_time
    
    if debug:
        print(f"Original columns: {df.columns.tolist()}")
        print(f"Processed columns: {result_df.columns.tolist()}")
        print(f"Schema columns: {list(db_schema.keys())}")
        
    return result_df

In [ ]:
#| export

@patch_to(SnowflakeConnector)    
def get_user_interactions(self, user_id: str, limit: int = 100) -> List[Dict[str, Any]]:
    """Get recent interactions for a specific user.
    
    Args:
        user_id: Slack user ID
        limit: Maximum number of records to return
        
    Returns:
        List of interaction records
    """
    query = f"""
        SELECT * 
        FROM SLACK_INTERACTIONS 
        WHERE USER_ID = %s 
        ORDER BY TIMESTAMP DESC 
        LIMIT {limit}
    """
    return self.execute_query(query, [user_id])

@patch_to(SnowflakeConnector)
def get_interactions_by_view(self, view: str, limit: int = 100) -> List[Dict[str, Any]]:
    """Get recent interactions for a specific view.
    
    Args:
        view: View name
        limit: Maximum number of records to return
        
    Returns:
        List of interaction records
    """
    query = f"""
        SELECT * 
        FROM SLACK_INTERACTIONS 
        WHERE VIEW = %s 
        ORDER BY TIMESTAMP DESC 
        LIMIT {limit}
    """
    return self.execute_query(query, [view])

@patch_to(SnowflakeConnector)
def get_interaction_summary(self) -> List[Dict[str, Any]]:
    """Get a summary of interactions by view and action type.
    
    Returns:
        Summary statistics
    """
    query = """
        SELECT 
            VIEW,
            ACTION_TYPE,
            COUNT(*) AS INTERACTION_COUNT,
            COUNT(DISTINCT USER_ID) AS UNIQUE_USERS,
            MIN(TIMESTAMP) AS FIRST_INTERACTION,
            MAX(TIMESTAMP) AS LATEST_INTERACTION
        FROM SLACK_INTERACTIONS
        GROUP BY VIEW, ACTION_TYPE
        ORDER BY VIEW, INTERACTION_COUNT DESC
    """
    return self.execute_query(query)

def __del__(self):
    """Ensure connection is closed when object is destroyed."""
    self.close()

# ActionHandler

In [ ]:
#| export

class ActionHandler:
    """
    Handles Slack interactive actions, including acknowledgment, response, 
    and storing results in Snowflake.
    
    Implements a singleton pattern to ensure only one handler exists.
    """
    
    # Singleton instance
    _instance = None
    
    @classmethod
    def get_instance(cls, snowflake_connector=None):
        """Get or create the singleton instance.
        
        Args:
            snowflake_connector: Optional Snowflake connector for the first initialization
            
        Returns:
            ActionHandler instance
        """
        if cls._instance is None:
            cls._instance = cls(snowflake_connector)
        return cls._instance
    
    def __init__(self, snowflake_connector):
        """Initialize with a Snowflake connector.
        
        Args:
            snowflake_connector: Object with methods to interact with Snowflake
        """
        if ActionHandler._instance is not None:
            raise Exception("This class is a singleton. Use get_instance() instead.")
            
        self.snowflake = snowflake_connector
        ActionHandler._instance = self
    
    @staticmethod
    def get_payload_from_body(body: Dict[str, Any]) -> Dict[str, Any]:
        """Extract relevant payload information from a Slack action body.
        
        Args:
            body: Slack action body
            
        Returns:
            Dictionary with standardized payload information
        """
        action = body["actions"][0]
        action_id = action["action_id"]
        print(f'Actiob Body: {body}')
        print(f'Action ID: {action_id}')
        print(f'Action: {action}')
        # Get common fields
        payload = {
            "user_id": body["user"]["id"],
            "user_name": body["user"]["name"],
            "channel_id": body["channel"]["id"],
            "message_ts": body["message"]["ts"],
            "action_id": action_id,
            "action_type": action["type"],
            "response_url": body["response_url"]
        }
        
        # Add type-specific data
        if action["type"] == "button":
            payload["value"] = action.get("value", "")
            payload["text"] = action["text"]["text"]
            
        elif action["type"] == "static_select" or action["type"] == "multi_static_select":
            if "selected_option" in action:
                payload["value"] = action["selected_option"]["value"]
                payload["text"] = action["selected_option"]["text"]["text"]
            elif "selected_options" in action:
                payload["values"] = [option["value"] for option in action["selected_options"]]
                payload["texts"] = [option["text"]["text"] for option in action["selected_options"]]
                
        elif action["type"] == "datepicker":
            payload["selected_date"] = action.get("selected_date", "")
            
        elif action["type"] == "users_select":
            payload["selected_user"] = action.get("selected_user", "")
            
        elif action["type"] == "channels_select":
            payload["selected_channel"] = action.get("selected_channel", "")
            
        return payload

### Responding to Actions:

In [ ]:
#| export

@patch_to(ActionHandler)
def _format_response_text(self, template: str, action_data: Dict[str, Any]) -> str:
    """Format response text by replacing placeholders.
    
    Args:
        template: Response text template
        action_data: Action data with values
        
    Returns:
        Formatted response text
    """
    # Make a copy that we'll modify
    result = template
    
    # Replace standard placeholders
    result = result.replace("{user}", action_data["user_name"])
    result = result.replace("{user_id}", action_data["user_id"])
    result = result.replace("{channel}", action_data["channel_id"])
    
    # Action-specific replacements
    if "value" in action_data:
        result = result.replace("{value}", str(action_data["value"]))
    
    if "text" in action_data:
        result = result.replace("{text}", str(action_data["text"]))
        
    if "selected_date" in action_data:
        date_str = action_data["selected_date"]
        try:
            # Try to format the date nicely
            date_obj = datetime.strptime(date_str, "%Y-%m-%d")
            formatted_date = date_obj.strftime("%B %d, %Y")
            result = result.replace("{date}", formatted_date)
        except:
            # Fallback to raw date
            result = result.replace("{date}", date_str)
            
    if "selected_user" in action_data:
        result = result.replace("{selected_user}", str(action_data["selected_user"]))
        
    if "selected_channel" in action_data:
        result = result.replace("{selected_channel}", str(action_data["selected_channel"]))
        
    # For multi-select values, join them
    if "values" in action_data:
        values_str = ", ".join(str(v) for v in action_data["values"])
        result = result.replace("{values}", values_str)
        
    if "texts" in action_data:
        texts_str = ", ".join(str(t) for t in action_data["texts"])
        result = result.replace("{texts}", texts_str)    
    return result

In [ ]:
#| export

@patch_to(ActionHandler)
def _send_response(self, action_data: Dict[str, Any], respond):
    """Send an appropriate response based on the action data.
    
    Args:
        action_data: Processed action data
        respond: Slack respond function or response URL
    """
    # Get view info and response config
    view_info = action_data["view_info"]
    response_config = view_info.get("response_config", {})
    
    # Default response text
    response_text = response_config.get("response_message", "Thank you for your response!")
    
    # Get response_type
    response_type = response_config.get("response_type", "ephemeral")
    
    # Replace placeholders with actual values
    response_text = self._format_response_text(response_text, action_data)
    
    # Prepare response payload
    response_payload = {
        "text": response_text,
        "replace_original": response_config.get("replace_original", False),
        "response_type": response_type
    }
    
    # Check if respond is a function or a URL
    if callable(respond):
        # It's a function, call it with kwargs
        respond(**response_payload)
    elif isinstance(respond, str) and respond.startswith("http"):
        # It's a response URL, make a POST request
        try:
            requests.post(
                respond,
                json=response_payload,
                headers={"Content-Type": "application/json"}
            )
        except Exception as e:
            print(f"Error sending response: {e}")
    else:
        # Can't respond
        print(f"No valid way to respond. Respond: {respond}")

### Storing Action in Snowflake

When the user interacts with the bot, the action is stored in Snowflake. The action is stored in a table called `user_interaction`. The table has the following columns:

In [ ]:
#| export

@patch_to(ActionHandler)
def _store_action_in_snowflake(self, action_data: Dict[str, Any], 
                                view_info: Dict[str, Any],
                                table_name: str = "SLACK_INTERACTIONS"):
    """Store interaction data in Snowflake.
    
    Args:
        action_data: Action data to store
        view_info: View information extracted from metadata
        table_name: Snowflake table name
    """
    if not self.snowflake:
            print("Snowflake connector not available, skipping storage")
            return
            
    try:
        # Convert to format suitable for Snowflake
        snowflake_data = {
            "ACTION_ID": action_data["action_id"],
            "VIEW": view_info.get("view",view_info.get("view_name","unknown")),
            "VIEW_GROUP": view_info.get("view_group", "unknown"),
            "ACTION_TYPE": action_data["components"]["action_type"],
            "ACTION_INDEX": action_data["components"]["index"],
            "ACTION_METADATA": json.dumps(action_data["components"].get("metadata", {})),
            "USER_ID": action_data["user_id"],
            "USER_NAME": action_data["user_name"],
            "CHANNEL_ID": action_data["channel_id"],
            "MESSAGE_TS": action_data["message_ts"],
            "RESPONSE_VALUE": action_data.get("value", 
                            action_data.get("selected_date",
                            action_data.get("selected_user",
                            action_data.get("selected_channel", "")))),
            "RESPONSE_TEXT": action_data.get("text", ""),
            "TIMESTAMP": datetime.now().isoformat(),
            "RAW_PAYLOAD": json.dumps(action_data)
        }
        
        # If we have multiple values (from multi-select), store as JSON
        if "values" in action_data:
            snowflake_data["RESPONSE_VALUES"] = json.dumps(action_data["values"])
            snowflake_data["RESPONSE_TEXTS"] = json.dumps(action_data["texts"])
        
        # Use the connector to insert into Snowflake
        self.snowflake.insert_record(table_name, snowflake_data)
    except Exception as e:
        print(f"Error storing action in Snowflake: {e}")

We also need a way to get information about a view in Snowflake:

# ActionIdManager

In [ ]:
#| export

class ActionIdManager:
    """
    Manages action IDs for Slack interactive elements to ensure uniqueness
    and provide routing capabilities for action handlers.
    """

    # Store metadata references
    _metadata_store = {}
    _counter = 0
    
    # Format: type_index_ref
    # Example: btn_0_a1b2
    ACTION_ID_PATTERN = "{type}_{idx}_{ref}"
    
    # For parsing action IDs back into components
    ACTION_ID_REGEX = re.compile(r"(?P<type>[^_]+)_(?P<idx>\d+)(_(?P<ref>[a-z0-9]+))?")

    @classmethod
    def _get_ref_id(cls, metadata: Optional[Dict[str, Any]] = None) -> str:
        """Generate a short reference ID for metadata.
        
        Args:
            metadata: Metadata to store
            
        Returns:
            Short reference ID
        """
        if not metadata:
            return ""
            
        # Generate a unique reference
        cls._counter += 1
        ref_id = f"{hex(cls._counter)[2:]}_{hex(int(time.time()))[2:8]}"
        
        # Store metadata with reference
        cls._metadata_store[ref_id] = metadata
        
        return ref_id

    @classmethod
    def get_metadata(cls, ref_id: str) -> Optional[Dict[str, Any]]:
        """Retrieve metadata using reference ID.
        
        Args:
            ref_id: Reference ID
            
        Returns:
            Associated metadata or None if not found
        """
        return cls._metadata_store.get(ref_id)
    

    @classmethod
    def generate_action_id(cls, action_type: str, index: int = 0, 
                          metadata: Optional[Dict[str, Any]] = None) -> str:
        """Generate a compact, unique action ID for a Slack interactive element.
        
        Args:
            action_type: Type of action (e.g., 'button', 'datepicker')
            index: Index for multiple elements of same type
            metadata: Optional metadata dictionary for action handling
            
        Returns:
            Unique action ID string
        """
        # Abbreviate common action types
        type_abbr = {
            'button': 'btn',
            'datepicker': 'date',
            'static_select': 'sel',
            'multi_static_select': 'msel',
            'users_select': 'user',
            'channels_select': 'chan'
        }.get(action_type, action_type[:4])  # Use first 4 chars for other types
        
        # Generate reference for metadata
        ref = cls._get_ref_id(metadata) if metadata else ""
        
        # Create action ID
        if ref:
            return cls.ACTION_ID_PATTERN.format(
                type=type_abbr, idx=index, ref=ref
            )
        else:
            # Without metadata, use simpler format
            return f"{type_abbr}_{index}"
    
    @classmethod
    def parse_action_id(cls, action_id: str) -> Dict[str, Any]:
        """Parse an action ID back into its components.
        
        Args:
            action_id: Action ID to parse
            
        Returns:
            Dictionary with action components and metadata
        """
        match = cls.ACTION_ID_REGEX.match(action_id)
        if not match:
            return {
                "action_type": None,
                "index": 0,
                "metadata": None
            }
        
        components = match.groupdict()
        
        # Map abbreviated types back to full names
        type_map = {
            'btn': 'button',
            'date': 'datepicker',
            'sel': 'static_select',
            'msel': 'multi_static_select',
            'user': 'users_select',
            'chan': 'channels_select'
        }
        
        # Set action type from mapping or keep as is
        components["action_type"] = type_map.get(components["type"], components["type"])
        components["index"] = int(components["idx"])
        
        # Look up metadata if reference exists
        ref = components.get("ref")
        if ref:
            components["metadata"] = cls.get_metadata(ref)
            
        return components
        
    @staticmethod
    def extract_context_metadata(message_blocks: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Extract metadata from context blocks in a message.
        
        Args:
            message_blocks: List of blocks from a Slack message
            
        Returns:
            Dictionary with extracted metadata
        """
        metadata = {}
        
        # Look for any context blocks that might contain metadata
        for block in message_blocks:
            if block.get("type") == "context":
                # Try to extract structured metadata from context blocks
                for element in block.get("elements", []):
                    if element.get("type") == "mrkdwn":
                        text = element.get("text", "")
                        # Look for key-value pairs
                        matches = re.findall(r"(\w+): ([^|]+)", text)
                        for key, value in matches:
                            metadata[key.strip().lower()] = value.strip()
                            
        return metadata

Let's now update the `ActionHandler` class to use the new `ActionIDManager` class.

Let's create a method to allow the user to setup the action handler in their slackbot app, and this handler will need to use an instance of the `ActionIDManager` class to generate a unique action ID for each action. The action ID will be used to identify the action in the Snowflake table

In [ ]:
#| export

@patch_to(ActionHandler,cls_method=True)
def setup_slack_action_handler(self,app, snowflake_connector):
    """Set up a single Slack action handler with the Bolt app.
    
    Args:
        app: Slack Bolt app
        snowflake_connector: Connector for Snowflake operations
        
    Returns:
        Initialized ActionHandler instance
    """
    # Create the singleton action handler
    handler = ActionHandler.get_instance(snowflake_connector)
    ACTION_ID_PREFIX_REGEX = re.compile(r"^[a-z]{3,4}_\d+")
    
    # Register the catch-all action handler
    @app.action(ACTION_ID_PREFIX_REGEX)
    def handle_all_actions(ack, body, logger, respond):
        """Universal handler for all interactive elements."""
        # Always acknowledge receipt
        ack()
        
        try:
            # Validate body structure
            if not body:
                logger.error("Body is None")
                return
                
            if 'actions' not in body or not body['actions']:
                logger.error(f"No actions in body: {json.dumps(body)}")
                return
            
            # Extract action information - with additional logging
            action = body['actions'][0]
            logger.info(f"Action data: {json.dumps(action)}")
            
            if 'action_id' not in action:
                logger.error(f"No action_id in action: {json.dumps(action)}")
                return
                
            action_id = action['action_id']
            logger.info(f"Handling action: {action_id}")
            
            # Parse components with safety check
            components = ActionIdManager.parse_action_id(action_id)
            if not components:
                logger.error(f"Failed to parse action_id: {action_id}")
                return
                
            logger.info(f"Action components: {components}")
            
            # Process the action
            if handler is None:
                logger.error("Handler is None")
                return
                
            handler.process_action(body, respond, components)
            
        except Exception as e:
            logger.error(f"Error processing action: {str(e)}")
            logger.error(f"Action body: {json.dumps(body) if body else 'None'}")
            import traceback
            logger.error(f"Traceback: {traceback.format_exc()}")
    
    print("Registered universal action handler for all block actions")
    
    return handler

We also need to update the `ActionHandler` class to use the new `ActionIDManager` class. When processing actions from the app, it will need to call `parse_action_id` to get the information from the id

In [ ]:
#| export

@patch_to(ActionHandler)
def process_action(self, body: Dict[str, Any], respond: Callable, 
                    components: Optional[Dict[str, Any]] = None):
    """Process a Slack action event.
    
    Args:
        body: Slack event body
        respond: Slack respond function
        components: Optional pre-parsed action ID components
        
    Returns:
        Processed action data
    """
    # Extract action data
    action_data = self.get_payload_from_body(body)
    
    # Parse the action ID if components not provided
    if not components:  action_data["components"] = ActionIdManager.parse_action_id(action_data["action_id"])
    else:               action_data["components"] = components
    
    # Extract message blocks to get additional context
    message_blocks = body["message"]["blocks"]
    
    # Look for metadata in context blocks
    #context_metadata = ActionIdManager.extract_context_metadata(message_blocks)
    action_data["context_metadata"] = body['message']['metadata']['event_payload']
    
    # Try to parse metadata for view info
    view_info = body['message']['metadata']['event_payload']
    action_data["view_info"] = view_info
    
    # Store interaction in Snowflake
    self._store_action_in_snowflake(action_data, view_info)
    
    # Determine how to respond based on metadata
    self._send_response(action_data, respond)
    
    return action_data

# InteractionBuilder

In [ ]:
#| export

class InteractionBuilder:
    """
    Utility class for creating interactive Slack Block Kit elements.
    """
    pass

First, let's start at the top of the message, which would be the message's header:

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_button(self, text: str, action_id: str, url: Optional[str] = None, 
                    value: Optional[str] = None, style: Optional[str] = None) -> Dict[str, Any]:
    """Create a button element for Slack messages.
    
    Args:
        text: Button text
        action_id: Identifier for the button
        url: Optional URL for link buttons
        value: Optional value for the button
        style: Optional style - "primary" or "danger"
        
    Returns:
        Slack block kit button object
    """
    button = {
        "type": "button",
        "text": {
            "type": "plain_text",
            "text": text,
            "emoji": True
        },
        "action_id": action_id
    }
    
    if url:
        button["url"] = url
        
    if value:
        button["value"] = value
        
    if style and style in ["primary", "danger"]:
        button["style"] = style
        
    return button

In [ ]:
test_eq(InteractionBuilder.create_button('My Button','button_clicked'),
        {'type': 'button',
        'text': {'type': 'plain_text', 'text': 'My Button', 'emoji': True},
        'action_id': 'button_clicked'})

InteractionBuilder.create_button('My Button','button_clicked')

{'type': 'button',
 'text': {'type': 'plain_text', 'text': 'My Button', 'emoji': True},
 'action_id': 'button_clicked'}

We also need a way to turn the action elements into the full action section/block:

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_actions_block(self, elements: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Create an actions block for Slack messages.
    
    Args:
        elements: List of interactive elements
        
    Returns:
        Slack block kit actions object
    """
    return {"type": "actions","elements": elements}

In [ ]:
test_eq(
    InteractionBuilder.create_actions_block(
        InteractionBuilder.create_button('Another Button','button_2_clicked')),
    {'type': 'actions',
    'elements': {'type': 'button',
    'text': {'type': 'plain_text', 'text': 'Another Button', 'emoji': True},
    'action_id': 'button_2_clicked'}}
)


InteractionBuilder.create_actions_block(
    InteractionBuilder.create_button('Another Button','button_2_clicked')
)

{'type': 'actions',
 'elements': {'type': 'button',
  'text': {'type': 'plain_text', 'text': 'Another Button', 'emoji': True},
  'action_id': 'button_2_clicked'}}

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_datepicker(self, action_id: str, placeholder: str, 
                         initial_date: Optional[str] = None) -> Dict[str, Any]:
        """Create a date picker element.
        
        Args:
            action_id: Identifier for the date picker
            placeholder: Placeholder text
            initial_date: Optional initial date (YYYY-MM-DD format)
            
        Returns:
            Slack block kit datepicker object
        """
        datepicker = {
            "type": "datepicker",
            "action_id": action_id,
            "placeholder": {
                "type": "plain_text",
                "text": placeholder,
                "emoji": True
            }
        }
        
        if initial_date:
            datepicker["initial_date"] = initial_date
            
        return datepicker

In [ ]:
test_eq(InteractionBuilder.create_datepicker('start_date','Pick a Start Date'),
        {'type': 'datepicker',
            'action_id': 'start_date',
            'placeholder': {'type': 'plain_text',
            'text': 'Pick a Start Date',
            'emoji': True}})

InteractionBuilder.create_datepicker('start_date','Pick a Start Date')

{'type': 'datepicker',
 'action_id': 'start_date',
 'placeholder': {'type': 'plain_text',
  'text': 'Pick a Start Date',
  'emoji': True}}

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_static_select(self, action_id: str, placeholder: str, 
                        options: List[Tuple[str, str]]) -> Dict[str, Any]:
    """Create a dropdown select element.
    
    Args:
        action_id: Identifier for the select
        placeholder: Placeholder text
        options: List of (text, value) tuples for options
        
    Returns:
        Slack block kit static_select object
    """
    select = {
        "type": "static_select",
        "action_id": action_id,
        "placeholder": {
            "type": "plain_text",
            "text": placeholder,
            "emoji": True
        },
        "options": [
            {
                "text": {
                    "type": "plain_text",
                    "text": text,
                    "emoji": True
                },
                "value": value
            }
            for text, value in options
        ]
    }
    
    return select

In [ ]:
test_eq(InteractionBuilder.create_static_select('credit_action','Choose and Action',[('Issue Creidt','issue_credit'),('Ignore','ignore')]),
        {'type': 'static_select',
            'action_id': 'credit_action',
            'placeholder': {'type': 'plain_text',
            'text': 'Choose and Action',
            'emoji': True},
            'options': [{'text': {'type': 'plain_text',
                'text': 'Issue Creidt',
                'emoji': True},
            'value': 'issue_credit'},
            {'text': {'type': 'plain_text', 'text': 'Ignore', 'emoji': True},
            'value': 'ignore'}]})

InteractionBuilder.create_static_select('credit_action','Choose and Action',[('Issue Creidt','issue_credit'),('Ignore','ignore')])

{'type': 'static_select',
 'action_id': 'credit_action',
 'placeholder': {'type': 'plain_text',
  'text': 'Choose and Action',
  'emoji': True},
 'options': [{'text': {'type': 'plain_text',
    'text': 'Issue Creidt',
    'emoji': True},
   'value': 'issue_credit'},
  {'text': {'type': 'plain_text', 'text': 'Ignore', 'emoji': True},
   'value': 'ignore'}]}

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_multi_select(self, action_id: str, placeholder: str, 
                        options: List[Tuple[str, str]]) -> Dict[str, Any]:
    """Create a multi-select dropdown element.
    
    Args:
        action_id: Identifier for the multi-select
        placeholder: Placeholder text
        options: List of (text, value) tuples for options
        
    Returns:
        Slack block kit multi_static_select object
    """
    multi_select = {
        "type": "multi_static_select",
        "action_id": action_id,
        "placeholder": {
            "type": "plain_text",
            "text": placeholder,
            "emoji": True
        },
        "options": [
            {
                "text": {
                    "type": "plain_text",
                    "text": text,
                    "emoji": True
                },
                "value": value
            }
            for text, value in options
        ]
    }
    
    return multi_select

In [ ]:
test_eq(InteractionBuilder.create_multi_select('select_multiple','Select Option(s)',[('One','one'),('Two','two')]),
        {'type': 'multi_static_select',
            'action_id': 'select_multiple',
            'placeholder': {'type': 'plain_text',
            'text': 'Select Option(s)',
            'emoji': True},
            'options': [{'text': {'type': 'plain_text', 'text': 'One', 'emoji': True},
            'value': 'one'},
            {'text': {'type': 'plain_text', 'text': 'Two', 'emoji': True},
            'value': 'two'}]})

InteractionBuilder.create_multi_select('select_multiple','Select Option(s)',[('One','one'),('Two','two')])

{'type': 'multi_static_select',
 'action_id': 'select_multiple',
 'placeholder': {'type': 'plain_text',
  'text': 'Select Option(s)',
  'emoji': True},
 'options': [{'text': {'type': 'plain_text', 'text': 'One', 'emoji': True},
   'value': 'one'},
  {'text': {'type': 'plain_text', 'text': 'Two', 'emoji': True},
   'value': 'two'}]}

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_users_select(self, action_id: str, placeholder: str) -> Dict[str, Any]:
        """Create a user select element.
        
        Args:
            action_id: Identifier for the users select
            placeholder: Placeholder text
            
        Returns:
            Slack block kit users_select object
        """
        return {
            "type": "users_select",
            "action_id": action_id,
            "placeholder": {
                "type": "plain_text",
                "text": placeholder,
                "emoji": True
            }
        }

In [ ]:
test_eq(InteractionBuilder.create_users_select('selected_user','Select a User'),
        {'type': 'users_select',
        'action_id': 'selected_user',
        'placeholder': {'type': 'plain_text', 'text': 'Select a User', 'emoji': True}})

InteractionBuilder.create_users_select('selected_user','Select a User')

{'type': 'users_select',
 'action_id': 'selected_user',
 'placeholder': {'type': 'plain_text', 'text': 'Select a User', 'emoji': True}}

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_channels_select(self, action_id: str, placeholder: str) -> Dict[str, Any]:
    """Create a channel select element.
    
    Args:
        action_id: Identifier for the channels select
        placeholder: Placeholder text
        
    Returns:
        Slack block kit channels_select object
    """
    return {
        "type": "channels_select",
        "action_id": action_id,
        "placeholder": {
            "type": "plain_text",
            "text": placeholder,
            "emoji": True
        }
    }

In [ ]:
test_eq(InteractionBuilder.create_channels_select('selected_channel','Select a Channel'),
        {'type': 'channels_select',
        'action_id': 'selected_channel',
        'placeholder': {'type': 'plain_text',
        'text': 'Select a Channel',
        'emoji': True}})

InteractionBuilder.create_channels_select('selected_channel','Select a Channel')

{'type': 'channels_select',
 'action_id': 'selected_channel',
 'placeholder': {'type': 'plain_text',
  'text': 'Select a Channel',
  'emoji': True}}

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def detect_and_create_interactive_elements(
        self,
        option_names: List[str], 
        option_values: List[str], 
        action_type: Optional[str] = None,
        metadata: Optional[str] = None,
        view_info: Optional[Dict[str, Any]] = None
        ) -> List[Dict[str, Any]]:
    """Smartly detect and create appropriate interactive elements.
    
    Args:
        option_names: List of option names
        option_values: List of option values
        action_type: Optional explicit action type
        metadata: Optional metadata to include in action_id
        view_info: Optional view information for embedding in metadata
        
    Returns:
        List of interactive elements
    """
    
    # Smart action type detection if not explicitly set
    if not action_type:
        # Check for date-related names to use date picker
        date_keywords = ['date', 'when', 'calendar', 'schedule', 'deadline', 'due']
        has_date_keyword = any(keyword in ' '.join(option_names).lower() for keyword in date_keywords)
        
        # Check if we have multiple options for a select
        if len(option_names) > 3:
            action_type = 'static_select'
        elif has_date_keyword:
            action_type = 'datepicker'
        elif any('user' in name.lower() for name in option_names):
            action_type = 'users_select'
        elif any('channel' in name.lower() for name in option_names):
            action_type = 'channels_select'
        elif any(['multi' in ''.join(option_names).lower(), 
                    'select multiple' in ''.join(option_names).lower()]):
            action_type = 'multi_static_select'
        else:
            action_type = 'button'
    
    # If view_info is provided, embed it in the metadata
    if view_info and metadata:
        # Combine view_info with existing metadata
        try:
            meta_dict = json.loads(metadata) if isinstance(metadata, str) else metadata
            if isinstance(meta_dict, dict) and isinstance(view_info, dict):
                meta_dict.update(view_info)
                metadata = json.dumps(meta_dict)
        except:
            # If metadata isn't JSON, append view info to metadata
            view_info_str = json.dumps(view_info)
            metadata = f"{metadata}|{view_info_str}"
    elif view_info:
        # Just use view_info as metadata
        metadata = json.dumps(view_info)
    
    elements = []
    
    # Create appropriate interactive elements based on action_type
    if action_type == 'datepicker':
        # Create date picker
        initial_date = None
        if option_values and option_values[0]:
            try:
                # Try to parse as date if it looks like one
                parsed_date = pd.to_datetime(option_values[0])
                initial_date = parsed_date.strftime('%Y-%m-%d')
            except Exception:
                pass
        
        # Generate unique action_id with metadata
        action_id = ActionIdManager.generate_action_id(
            action_type=action_type,
            index=0,
            metadata=metadata
        )
        
        elements.append(
            InteractionBuilder.create_datepicker(
                action_id=action_id,
                placeholder=option_names[0],
                initial_date=initial_date
            )
        )
        
    elif action_type == 'static_select':
        # Create dropdown with options
        options = [(name, value) for name, value in zip(option_names, option_values)]
        
        # Generate unique action_id with metadata
        action_id = ActionIdManager.generate_action_id(
            action_type=action_type,
            index=0,
            metadata=metadata
        )
        
        elements.append(
            InteractionBuilder.create_static_select(
                action_id=action_id,
                placeholder="Select an option",
                options=options
            )
        )
        
    elif action_type == 'multi_static_select':
        # Create multi-select dropdown
        options = [(name, value) for name, value in zip(option_names, option_values)]
        
        # Generate unique action_id with metadata
        action_id = ActionIdManager.generate_action_id(
            action_type=action_type,
            index=0,
            metadata=metadata
        )
        
        elements.append(
            InteractionBuilder.create_multi_select(
                action_id=action_id,
                placeholder="Select options",
                options=options
            )
        )
        
    elif action_type == 'users_select':
        # Create user select
        # Generate unique action_id with metadata
        action_id = ActionIdManager.generate_action_id(
            action_type=action_type,
            index=0,
            metadata=metadata
        )
        
        elements.append(
            InteractionBuilder.create_users_select(
                action_id=action_id,
                placeholder=option_names[0] if option_names else "Select a user"
            )
        )
        
    elif action_type == 'channels_select':
        # Create channel select
        # Generate unique action_id with metadata
        action_id = ActionIdManager.generate_action_id(
            action_type=action_type,
            index=0,
            metadata=metadata
        )
        
        elements.append(
            InteractionBuilder.create_channels_select(
                action_id=action_id,
                placeholder=option_names[0] if option_names else "Select a channel"
            )
        )
        
    else:
        # Default to buttons
        for i, (name, value) in enumerate(zip(option_names, option_values)):
            # Generate unique action_id with metadata
            action_id = ActionIdManager.generate_action_id(
                action_type=action_type,
                index=i,
                metadata=metadata
            )
            
            elements.append(
                InteractionBuilder.create_button(
                    text=str(name),
                    action_id=action_id,
                    value=str(value),
                    style="primary" if i == 0 else None
                )
            )
    
    return elements

In [ ]:
test_eq(InteractionBuilder.detect_and_create_interactive_elements(['Start Date'],['start_date'],'selected_start_date'),
        [{'type': 'button',
            'text': {'type': 'plain_text', 'text': 'Start Date', 'emoji': True},
            'action_id': 'sele_0',
            'value': 'start_date',
            'style': 'primary'}])

InteractionBuilder.detect_and_create_interactive_elements(['Start Date'],['start_date'],'selected_start_date')

[{'type': 'button',
  'text': {'type': 'plain_text', 'text': 'Start Date', 'emoji': True},
  'action_id': 'sele_0',
  'value': 'start_date',
  'style': 'primary'}]

#| hide

# Export
This is mainly here just to allow me to fold sections above here and still have this section visible

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()